In [1]:
import obspy
import struct
import numpy as np
import math
import array
import matplotlib
import matplotlib.pyplot as plt
from pathlib import Path
print('Previous matplotlib backend: ' + matplotlib.get_backend())
%matplotlib qt5
print('Current matplotlib backend: ' + matplotlib.get_backend())


def _bcd(byte):
    """Decode 1-byte binary code decimals."""

    if isinstance(byte, str):
        try:
            byte = ord(byte)
        except TypeError:
            raise ValueError('not a byte')
    elif isinstance(byte, int):
        if byte > 255:
            raise ValueError('not a byte')

    else:
        raise ValueError('not a byte')
    v1 = byte >> 4
    v2 = byte & 0xF
    return v1, v2

def _decode_bcd(bytes_in):
    """Decode arbitrary length binary code decimals."""
    v = 0
    if isinstance(bytes_in, int):
        bytes_in = bytes([bytes_in])
        disp('bytes in')
    n = len(bytes_in)
    n = n*2 - 1  # 2 values per byte
    for byte in bytes_in:
        v1, v2 = _bcd(byte)
        v += v1*10**n + v2*10**(n-1)
        n -= 2
    return v

def _decode_bin(bytes_in):
    """Decode unsigned ints."""
    if isinstance(bytes_in, int):
        bytes_in = bytes([bytes_in])
    ll = len(bytes_in)
    # zero-pad to 4 bytes
    b = (chr(0)*(4-ll)).encode()
    b += bytes_in
    return struct.unpack('>I', b)[0]

def _decode_flt(bytes_in):
    """Decode single-precision floats."""
    if isinstance(bytes_in, int):
        bytes_in = bytes([bytes_in])
    ll = len(bytes_in)
    # zero-pad to 4 bytes
    b = (chr(0)*(4-ll)).encode()
    b += bytes_in
    f = struct.unpack('>f', b)[0]
    if math.isnan(f):
        f = None
    return f

def _decode_fraction(bytes_in):
    """Decode positive binary fractions."""
    bytes_ord = bytes_in
    bit = ''.join('{:08b}'.format(b) for b in bytes_ord)
    return sum(int(x) * 2**-n for n, x in enumerate(bit, 1))

Previous matplotlib backend: module://matplotlib_inline.backend_inline
Current matplotlib backend: Qt5Agg


In [3]:
smpl_file = r'.\data\vibe0000008.sgd'
fp = open(smpl_file, 'rb')
buf_header1 = fp.read(32)
buf_header2 = fp.read(32)
fp.close()

In [14]:
general_header1 = dict()
if _decode_bcd(buf_header1[0:2]) > 9999:
    general_header1['file number'] = '{0:0{1}x}'.format(buf_header1[0], 2) + '{0:0{1}x}'.format(buf_header1[1], 2)
else:
    general_header1['file number'] = _decode_bcd(buf_header1[0:2])
general_header1['format code'] = _decode_bcd(buf_header1[2:4])
_year = _decode_bcd(buf_header1[10:11]) + 2000
_nblocks, _jday = _bcd(buf_header1[11])
general_header1['n_additional_blocks'] = _nblocks   # important
_jday *= 100
_jday += _decode_bcd(buf_header1[12:13])
_hour = _decode_bcd(buf_header1[13:14])
_min = _decode_bcd(buf_header1[14:15])
_sec = _decode_bcd(buf_header1[15:16])
general_header1['time'] = obspy.UTCDateTime(year=_year, julday=_jday,
                               hour=_hour, minute=_min, second=_sec)
general_header1['base_scan_interval'] = _decode_bcd(buf_header1[22:23])
general_header1['record_length'] = _decode_bcd(buf_header1[26:27])
general_header1['scan_type_per_record'] = _decode_bcd(buf_header1[27:28])
general_header1['n_channel_sets_per_scan_type'] = _decode_bcd(buf_header1[28:29])

In [15]:
general_header1

{'file number': 8,
 'format code': 8058,
 'n_additional_blocks': 1,
 'time': 2019-02-20T16:55:48.000000Z,
 'base_scan_interval': 8,
 'record_length': 165,
 'scan_type_per_record': 1,
 'n_channel_sets_per_scan_type': 3}

In [16]:
buf_header1[26:27]

b'\xff'

In [25]:
buf_header1[22:23]

b'\x08'

In [39]:
f'{0x08:0>8b}'
f'{0x08:0<8b}'

'10000000'

In [26]:
_decode_bcd(buf_header1[22:23])

8

In [24]:
_decode_fraction(buf_header1[22:23])

0.03125

In [35]:
bin(buf_header1[22:23])

TypeError: 'bytes' object cannot be interpreted as an integer

In [44]:
buf_header1[25:26].format(int(x,8))

AttributeError: 'bytes' object has no attribute 'format'

In [42]:
buf_header1[24:25]

b'\x00'

In [54]:
_bcd(buf_header1[25])

(8, 15)

In [52]:
_bcd(buf_header1[23])

(0, 0)

In [49]:
_bcd(buf_header1[22])

(0, 8)

In [51]:
# 아마 이게 맞을 듯?
_bsi = _decode_bcd(buf_header1[22:23])
if _bsi < 10:
    _bsi = 1./_bsi
else:
    _bsi /= 10.
_bsi

0.125

In [55]:
_record_types = {
    8: 'normal',
    2: 'test record'
}
_rec_type, _rec_len = _bcd(buf_header1[25])
record_type = _record_types[_rec_type]
_rec_len = 0x100 * _rec_len
_rec_len += _decode_bin(buf_header1[26:27])
if _rec_len == 0xFFF:
    _rec_len = None

In [56]:
record_type

'normal'

In [60]:
_rec_len